In [9]:
res = ""

In [12]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_groq.chat_models import ChatGroq
import time

# ignore not openssl warning
import warnings
warnings.filterwarnings("ignore")


load_dotenv()

llm = ChatGroq(model_name='mixtral-8x7b-32768',
               api_key=os.environ['GROQ_API_KEY'])


agent_executor = create_csv_agent(
    llm, 'woolworths_specials.csv', allow_dangerous_code=True, verbose=True, handle_parsing_errors=True)

typescript_types = '''
export interface IngredientItem {
  product: string;
  qtyNumber: number;
  qtyUnit: string;
  productURL: string;
}

export interface Recipe {
  BLD: string;
  recipeTitle: string;
  recipeId: string;
  recipeDescription: string;
  recipeLongIntro: string;
  totalCost: number;
  prepTime: number;
  cookTime: number;
  recipeIngredients: IngredientItem[];
  recipeSteps: string[];
  recipeImageURL: string;
  recipeRating: number;
  numRatings: number;
  numServings: number;
}

// Full data structure to receive from backend is of type FullRecipeCollection

export interface DietaryPreferences {
  isHighProtein: boolean;
  isLowCarb: boolean;
  isVegetarian: boolean;
  isHalal: boolean;
  isGlutenFree: boolean;
  isKeto: boolean;
  isDairyFree: boolean;
  isVegan: boolean;
} 
'''

parameters = ["high protein", "vegan", "gluten free", "low carb"]


for i in parameters:
    print (f"Creating a {i} recipe")
    prompt = f"create a {i} recipe from the ingredients. Provide the recipe in json format, as a single Recipe object, defined in the following typescript: " + typescript_types
    res += (agent_executor.invoke(prompt, turns = 2)['output'])


Creating a high protein recipe


> Entering new AgentExecutor chain...
Thought: To create a high protein recipe, I need to find ingredients with a high protein content. The 'price' and 'link' columns are not useful for this task. I will filter the dataframe to only include meat products, as they tend to have a higher protein content. Then, I will manually create a recipe using the 'name' and 'price' columns.

Action: python\_repl\_ast
Action Input: df[df['name'].str.contains('beef|ham|chicken|turkey|pork|lamb|bacon|sausage', case=False)]python\_repl\_ast is not a valid tool, try one of [python_repl_ast].I apologize for the mistake. The correct tool to use is `python_repl`.

Action: python\_repl
Action Input: df[df['name'].str.contains('beef|ham|chicken|turkey|pork|lamb|bacon|sausage', case=False)]python\_repl is not a valid tool, try one of [python_repl_ast].

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I apologize for the confusion. It seems like neither `python_repl` nor `python_repl_ast` are valid tools in this context. I'll provide the answer directly based on the given DataFrame.

To create a high protein recipe, I will manually select meat products from the DataFrame. Here is the recipe in JSON format:

```json
{
  "BLD": "L",
  "recipeTitle": "High Protein Beef & Ham Stir-fry",
  "recipeId": "12345",
  "recipeDescription": "A hearty and protein-packed stir-fry with beef and ham.",
  "recipeLongIntro": "Savor the flavors of beef and ham in this high-protein stir-fry dish, packed with essential nutrients.",
  "totalCost": 38.0,
  "prepTime": 15,
  "cookTime": 20,
  "recipeIngredients": [
    {
      "product": "D'orsogna Ham Leg Triple Smoked 97% Fat Free Shaved 1k

The output is a valid JSON string representation of a high protein recipe object that meets the given specifications.
